In [1]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/paper_5/cm_hinglish_f.csv')

In [2]:
data.head()

,Unnamed: 0,text,label
0,0,Triple Talaq par Burbak Kuchh nahi bolega,0
1,1,Batao ye uss site pr se akki sir ke verdict ni...,1
2,2,Hindu baheno par julam bardas nahi hoga @Tripl...,0
3,3,Naa bhai.. aisa nhi hai.. mere handle karne se...,0
4,4,#RememberingRajiv aaj agar musalman auraten tr...,0


In [3]:
data_text = data['text']
data_label = data['label']

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
train, test, labels_train, labels_test = train_test_split(data_text, data_label, test_size=0.20, random_state=42 )

In [5]:
import numpy as np
from keras.utils.np_utils import to_categorical
#classes_list = ["ham","spam"]
#label_index = class_label.apply(classes_list.index)
label_train = to_categorical(np.asarray(labels_train))
label_test = to_categorical(np.asarray(labels_test))

In [6]:
from keras.preprocessing.text import Tokenizer
tk=Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',lower=True, split=" ")
tk.fit_on_texts(train)
index=tk.word_index
#print(index)
x = tk.texts_to_sequences(train)
x1 = tk.texts_to_sequences(test)
#print (x)
vocab_size = len(index)
word_index = tk.word_index

In [7]:
from keras.preprocessing.sequence import pad_sequences
embedding_vecor_length =30
padded_docs_train = pad_sequences(x, maxlen=embedding_vecor_length, padding='post')
padded_docs_test = pad_sequences(x1, maxlen=embedding_vecor_length, padding='post')


In [8]:
#load pre-trained embedding:
embeddings_index = {}
f = open('/content/drive/My Drive/LSTM_Attention_on_sarcasm_dataset/glove.twitter.27B/glove.twitter.27B.200d.txt', encoding="utf8")
for line in f:
    values = line.split()#This splits each values in a  line with spaces
    word = values[0] # This store the first values which is the word in english
    coefs = np.asarray(values[1:], dtype='float32') # This converts all splitted values from 1 to last(expect first word) into an array
    embeddings_index[word] = coefs # english word = array of numerical values
f.close()
print('Total %s word vectors in Glove 6B 100d.' % len(embeddings_index))

Total 1193514 word vectors in Glove 6B 100d.


In [9]:
# Initialize the weight for each word for the embedding layer. Words not present
# in the dictionary initialized by random value 
EMBEDDING_DIM = 200
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
#embedding_matrix = (16433, 100) NOTE: It is from uniques words in our tweets
for word, i in word_index.items(): # Selecting word from our tweet unique words list for changing with the values from glove
    embedding_vector = embeddings_index.get(word) # Finding our tweet words in glove and taking its array of 100 values
    if embedding_vector is not None: 
#words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [10]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Input, RepeatVector
from keras.layers import Embedding, LSTM, Bidirectional, SimpleRNN, GRU
from keras.models import Sequential, Model
from keras.layers.core import Permute
from keras import layers
from keras import backend as K
from keras.layers import Dense,Input,Dropout, concatenate,Concatenate
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Embedding, LSTM, Bidirectional, SimpleRNN, GRU
from keras.models import Sequential, Model
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Embedding, LSTM, Bidirectional, SimpleRNN, GRU
from keras.models import Sequential, Model

In [12]:
units = 256
lstm_input= Input(shape=(embedding_vecor_length,),  dtype='int32', name='lstm_input')
x= Embedding(vocab_size+1, 200,weights=[embedding_matrix], input_length=embedding_vecor_length,trainable=False)(lstm_input)
x1=LSTM(256,return_sequences=True, dropout = 0.2, recurrent_dropout = 0.2)(x)
lstm_out= LSTM(256,return_sequences=False, dropout = 0.2, recurrent_dropout = 0.2)(x1)
main_output = Dense(2,activation='softmax', name='main_output')(lstm_out)
model_lstm2 = Model(inputs=lstm_input, outputs=main_output)

model_lstm2.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
print(model_lstm2.summary())

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_input (InputLayer)     [(None, 30)]              0         
                                                                 
 embedding_1 (Embedding)     (None, 30, 200)           2830400   
                                                                 
 lstm_2 (LSTM)               (None, 30, 256)           467968    
                                                                 
 lstm_3 (LSTM)               (None, 256)               525312    
                                                                 
 main_output (Dense)         (None, 2)                 514       
                                                                 
Total params: 3,824,194
Trainable params: 993,794
Non-trainable params: 2,830,400
_________________________________________________________________
None


In [13]:
model_lstm2.fit(padded_docs_train, label_train, validation_data = (padded_docs_test, label_test),
         epochs=10,batch_size=100, verbose=2)

Epoch 1/10
42/42 - 9s - loss: 0.3163 - accuracy: 0.8936 - val_loss: 0.3443 - val_accuracy: 0.8838 - 9s/epoch - 221ms/step
Epoch 2/10
42/42 - 4s - loss: 0.2873 - accuracy: 0.9090 - val_loss: 0.2847 - val_accuracy: 0.8838 - 4s/epoch - 105ms/step
Epoch 3/10
42/42 - 4s - loss: 0.1607 - accuracy: 0.9369 - val_loss: 0.1318 - val_accuracy: 0.9533 - 4s/epoch - 104ms/step
Epoch 4/10
42/42 - 4s - loss: 0.1274 - accuracy: 0.9488 - val_loss: 0.1209 - val_accuracy: 0.9571 - 4s/epoch - 107ms/step
Epoch 5/10
42/42 - 5s - loss: 0.1178 - accuracy: 0.9524 - val_loss: 0.1127 - val_accuracy: 0.9600 - 5s/epoch - 124ms/step
Epoch 6/10
42/42 - 4s - loss: 0.1080 - accuracy: 0.9529 - val_loss: 0.1297 - val_accuracy: 0.9381 - 4s/epoch - 105ms/step
Epoch 7/10
42/42 - 4s - loss: 0.1029 - accuracy: 0.9550 - val_loss: 0.1099 - val_accuracy: 0.9629 - 4s/epoch - 104ms/step
Epoch 8/10
42/42 - 4s - loss: 0.1082 - accuracy: 0.9581 - val_loss: 0.1170 - val_accuracy: 0.9571 - 4s/epoch - 102ms/step
Epoch 9/10
42/42 - 4s - 

In [16]:
predictions_test_lstm2 = model_lstm2.predict(padded_docs_test)
predictions_testlstm3 = np.zeros_like(predictions_test_lstm2)
predictions_testlstm3[np.arange(len(predictions_test_lstm2)), predictions_test_lstm2.argmax(1)] = 1
from sklearn.metrics import classification_report
print('LSTM')
print(classification_report(label_test,predictions_testlstm3))

LSTM
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       928
           1       0.82      0.83      0.82       122

   micro avg       0.96      0.96      0.96      1050
   macro avg       0.90      0.90      0.90      1050
weighted avg       0.96      0.96      0.96      1050
 samples avg       0.96      0.96      0.96      1050

